# Table of Contents
* [Submitting HITs](#Submitting-HITs)
	* [Building URLs for images on s3](#Building-URLs-for-images-on-s3)
	* [submitting HITs in groups](#submitting-HITs-in-groups)
		* [creates HITs, careful with this one](#creates-HITs,-careful-with-this-one)
* [Reviewing latest HITs](#Reviewing-latest-HITs)
* [Merging latest round of HITs into combined dataset](#Merging-latest-round-of-HITs-into-combined-dataset)
	* [Load prior complete dataset if not in memory](#Load-prior-complete-dataset-if-not-in-memory)
	* [Updating full dataset](#Updating-full-dataset)
	* [Working with full dataset](#Working-with-full-dataset)
* [Worker Analysis](#Worker-Analysis)
	* [Basic worker stats](#Basic-worker-stats)
		* [HIT duration for pricing](#HIT-duration-for-pricing)
		* [Identifying high and low consensus workers](#Identifying-high-and-low-consensus-workers)
	* [Messaging workers](#Messaging-workers)
* [HIT end-of-life](#HIT-end-of-life)
	* [Pickle latest results](#Pickle-latest-results)
	* [Pickle combined dataset](#Pickle-combined-dataset)
	* [Accepting and deleting HITs... careful with these](#Accepting-and-deleting-HITs...-careful-with-these)


In [77]:
%%capture
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict
%load_ext autoreload
%autoreload 2

#The lines commented below set the look and feel of mpl generated plots.
import matplotlib as mpl
# mpl.use("Agg")
# import matplotlib.pylab as plt
# #%matplotlib notebook
# %matplotlib inline
# %load_ext base16_mplrc
# %base16_mplrc light default
# plt.rcParams['figure.figsize'] = (16.0, 10.0)

import re
import pickle
import boto
import json
import os

from copy import deepcopy
import boto.mturk.connection as tc
import boto.mturk.question as tq
from boto.mturk.qualification import PercentAssignmentsApprovedRequirement, Qualifications, Requirement

# These lines import my aws access keys
from keysTkingdom import mturk_ai2
from keysTkingdom import aws_tokes


import amt_utils.process_hits as amt_util
import amt_utils.turk_email_utils as turkmail_util
import amt_utils.shining_rel_build as rbuild
import glob

# **Submitting HITs**

## Building URLs for images on s3

In [78]:
images_to_annotate = pd.read_csv('./list_to_annotate.txt', header=None)
images_to_annotate.columns= ['image_name']
images_to_annotate_list = images_to_annotate['image_name'].tolist()
len(images_to_annotate_list)

400

In [91]:
with open('dim_lookup.pkl',) as f:
    img_dim_lookup = pickle.load(f)
with open('bad_images.pkl',) as f:
    bad_images = pickle.load(f)

In [107]:
bad_images
bad_pages = [app_url_base + page + '&hd=' + str(img_dim_lookup[page][0]) +'&vd=' + str(img_dim_lookup[page][1]) for page in bad_images]

In [108]:
%page bad_pages

In [99]:
app_url_base = 'https://s3-us-west-2.amazonaws.com/ai2-vision-turk-data/shining-3-watercycle-test/build/index.html?url='

pages_to_use = [app_url_base + page + '&hd=' + str(img_dim_lookup[page][0]) +'&vd=' + str(img_dim_lookup[page][1]) for page in images_to_annotate_list]
# pages_to_use = [app_url_base + page + '&hd=' + str(img_dim_lookup[page][0]) +'&vd=' + str(img_dim_lookup[page][1]) for page in bad_images]

In [105]:
pages_to_use_sub = pages_to_use[151:152]

## submitting HITs in groups

**DON'T FORGET to change submission POST request in the client when changing host**

In [102]:
## Switch between sandbox and the real world here ##
## DON'T FORGET to change submission POST request in the client ##

sandbox_host = 'mechanicalturk.sandbox.amazonaws.com' 
real_world_host = 'mechanicalturk.amazonaws.com'
mturk = tc.MTurkConnection(
    aws_access_key_id = aws_tokes.access_key,
    aws_secret_access_key = aws_tokes.access_secret_key,
    host = sandbox_host,
    debug = 1 # debug = 2 prints out all requests.
)
current_account_balance = mturk.get_account_balance()[0]
if current_account_balance.amount == 10000:
    print "Working in the SANDBOX with"
else:
    print "Working in the REAL WORLD with"
print current_account_balance # a reminder of sandbox

Working in the SANDBOX with
$10,000.00


In [103]:
static_params = {
    'title': "Annotate shining 3 water cycles",
    'description': "you know what to do",
    'keywords': ['findme'],
    'frame_height': 1000,
    'amount': 1.0,
    'duration': 3600 * 12,
    'lifetime': 3600 * 24 * 3,
    'max_assignments': 1   # change to 3 when running for real
}

expected_cost = len(pages_to_use_sub) *  static_params['amount'] * static_params['max_assignments']
if float(current_account_balance.amount) < expected_cost:
    print('WARNING -- account balance is too low -- WARNING')
print('expect this batch of HITs to cost: $' + str(expected_cost))

expect this batch of HITs to cost: $1.0


### creates HITs, careful with this one

**uncomment cell below only when ready to submit**

In [106]:
amt_util.create_hits_from_pages(mturk, pages_to_use_sub, static_params)

## Qualifications

In [34]:
new_qual_name = 'science textbook annotation'
new_qual_description = 'Qualification for workers who have proved to be good at annotating science textbooks'
# my_qual_type = mturk.create_qualification_type(new_qual_name, new_qual_description, status='Active')

my_qual_type = mturk.search_qualification_types(new_qual_name)[0]

In [55]:
my_qual_id = my_qual_type.QualificationTypeId
print my_qual_id

In [ ]:
mturk.assign_qualification(my_qual_id, worker_id)

# **Reviewing HITs**

### Retrieving and processing latest HITs

In [109]:
r_hits_current_batch = amt_util.get_completed_hits(mturk)
assignment_results_current_batch = amt_util.get_assignments(mturk, r_hits_current_batch, 'Submitted')

In [110]:
raw_hit_results_current_batch = amt_util.process_raw_hits(assignment_results_current_batch)

In [111]:
amt_util.get_assignment_statuses(assignment_results_current_batch )

Submitted    2
dtype: int64

The commands above interact with mechanical turk and can take a while

In [112]:
results_df_shining_batch = amt_util.make_shining_results_df(raw_hit_results_current_batch)

In [114]:
results_df_shining_batch

,page,category,hit_id,assignment_id,id,worker_id,group_n
0,2798.png,[unlabeled],3HO4MYYR12PH0HGR4ND12NK3VZAU68,36U2A8VAG1083PJF6U9T4BQF674KYW,T10,AO235WMFCB1D9,"[[0, 0]]"
1,2798.png,"[unlabeled, IntraObjectLabel]",3HO4MYYR12PH0HGR4ND12NK3VZAU68,36U2A8VAG1083PJF6U9T4BQF674KYW,T11,AO235WMFCB1D9,"[[0, 0], [12, 1]]"
2,2798.png,"[unlabeled, IntraObjectLabel]",3HO4MYYR12PH0HGR4ND12NK3VZAU68,36U2A8VAG1083PJF6U9T4BQF674KYW,T12,AO235WMFCB1D9,"[[0, 0], [13, 2]]"
3,2798.png,[unlabeled],3HO4MYYR12PH0HGR4ND12NK3VZAU68,36U2A8VAG1083PJF6U9T4BQF674KYW,A10,AO235WMFCB1D9,"[[0, 0]]"
4,2798.png,"[unlabeled, IntraObjectLinkage]",3HO4MYYR12PH0HGR4ND12NK3VZAU68,36U2A8VAG1083PJF6U9T4BQF674KYW,A1,AO235WMFCB1D9,"[[0, 0], [8, 2]]"
5,2798.png,"[unlabeled, IntraObjectLinkage]",3HO4MYYR12PH0HGR4ND12NK3VZAU68,36U2A8VAG1083PJF6U9T4BQF674KYW,A0,AO235WMFCB1D9,"[[0, 0], [7, 2]]"
6,2798.png,"[unlabeled, IntraObjectLinkage, arrowDescriptor]",3HO4MYYR12PH0HGR4ND12NK3VZAU68,36U2A8VAG1083PJF6U9T4BQF674KYW,A3,AO235WMFCB1D9,"[[0, 0], [9, 2], [19, 1]]"
7,2798.png,"[unlabeled, IntraObjectLinkage, arrowDescriptor]",3HO4MYYR12PH0HGR4ND12NK3VZAU68,36U2A8VAG1083PJF6U9T4BQF674KYW,A2,AO235WMFCB1D9,"[[0, 0], [10, 2], [18, 2]]"
8,2798.png,[unlabeled],3HO4MYYR12PH0HGR4ND12NK3VZAU68,36U2A8VAG1083PJF6U9T4BQF674KYW,A5,AO235WMFCB1D9,"[[0, 0]]"
9,2798.png,"[unlabeled, IntraObjectLinkage]",3HO4MYYR12PH0HGR4ND12NK3VZAU68,36U2A8VAG1083PJF6U9T4BQF674KYW,A4,AO235WMFCB1D9,"[[0, 0], [11, 2]]"


In [118]:
anno_dir = './simpleAnnotations/'
new_anno_dir = './newAnnotations/'
test_image_file = '1311.png'
test_anno_file = test_image_file + '.json'

In [132]:
_ = results_df_shining_batch.groupby('page').apply(rbuild.build_and_write_relationships, anno_dir, new_anno_dir)

### consensus

In [237]:
# results_df_question_batch['group_memberships'] = results_df_question_batch['g_list'].apply(lambda x:[el[0] for el in x if el[0] > 0])

In [158]:
# consensus_results_df_current_batch = amt_util.make_consensus_df(results_df_question_batch, 'No Consensus')

In [319]:
question_cats = ['Multiple Choice',
                 'Fill-in-the-Blank',
                 'Short Answer',
                 'Discussion']

In [332]:
question_only_cons_df = consensus_results_df_current_batch[consensus_results_df_current_batch['box_id'].apply(lambda x:x[0] == 'Q' ) | consensus_results_df_current_batch['category'].isin(question_cats)]

In [312]:
no_consensus_hits_cat = consensus_results_df_current_batch[consensus_results_df_current_batch['category'] == 'No Consensus']

flaw_rate = len(no_consensus_hits_cat) / len(consensus_results_df_current_batch)
print 'question boxes without category consensus comprise ' + '{0:0.2f}% '.format(flaw_rate * 100) + 'of the total'  

These numbers differ because-
1. The first counts non-con results from the first round (non-question results)

2. The second counts only boxes either previously marked as a question, or selected in ths round

In [338]:
no_consensus_hits_cat = question_only_cons_df[question_only_cons_df['category'] == 'No Consensus']

flaw_rate = len(no_consensus_hits_cat) / len(question_only_cons_df)
print 'question boxes without category consensus comprise ' + '{0:0.2f}% '.format(flaw_rate * 100) + 'of the total'  

In [315]:
amt_util.write_results_df(consensus_results_df_current_batch, 'annotations-w-questions/', 'simpler-test-questions/')

In [317]:
# consensus_results_df_current_batch[consensus_results_df_current_batch['page'] == 'Spectrum_Science_Grade_8_8.jpeg']

## **sending to review tool**

### randomly sampling HITs 

In [375]:
pages_to_review = np.unique(consensus_results_df_current_batch['page'])

In [376]:
sampling_rate = 0.1
sample_size = int(len(pages_to_review) * sampling_rate)
sampled_pages_to_review = list(np.random.choice(pages_to_review, size= sample_size, replace=False))
print 'sampling ' + str(sample_size) + ' pages out of ' + str(len(pages_to_review))
to_review = ['start_seq'] + sampled_pages_to_review

In [377]:
anno_dir = 'simpler-test-questions/'
amt_util.review_results(to_review, anno_dir)
print 'posting to review tool, navigate to http://localhost:8080/ to see the sampled consensus results'

### sampling no consensus

In [378]:
pages_to_review =pd.unique(no_consensus_hits_cat['page'])

In [379]:
sampling_rate = 0.5
sample_size = int(len(pages_to_review) * sampling_rate)
sampled_pages_to_review = list(np.random.choice(pages_to_review, size= sample_size, replace=False))
print 'sampling ' + str(sample_size) + ' pages out of ' + str(len(pages_to_review))
to_review = ['start_seq'] + sampled_pages_to_review

In [380]:
anno_dir = 'simpler-test-questions/'
amt_util.review_results(to_review, anno_dir)
print 'posting to review tool, navigate to http://localhost:8080/ to see the sampled consensus results'

### looking at individual workers

In [276]:
ind_worker_dir = 'individual-worker-results/'

In [342]:
bad_and_prolific_to_review

In [384]:
# workers_to_review = turkers_contacted_me[:1]
workers_to_review = really_bad_workers_to_review
subset_by_worker = results_df_question_batch[results_df_question_batch['worker_id'].isin(workers_to_review)]

amt_util.write_results_df(subset_by_worker, 'annotations-w-questions/', ind_worker_dir)
pages_to_review = pd.unique(subset_by_worker['page']).tolist()
to_review = ['start_seq'] + pages_to_review 

In [383]:
anno_dir = ind_worker_dir
amt_util.review_results(to_review, anno_dir)
print 'posting to review tool, navigate to http://localhost:8080/ to see the sampled consensus results'

### store cells

In [207]:
multi_choice_pages = consensus_results_df_complete[consensus_results_df_complete['category'] == 'Multiple Choice']
pages_to_review =pd.unique(multi_choice_pages['page'])
to_review = ['start_seq'] + pages_to_review.tolist()

# consensus_results_df_complete = amt_util.make_consensus_df(results_df_question_batch, 'No Consensus')
consensus_results_df_complete = combined_consensus_df

# amt_util.write_results_df(consensus_results_df_complete, 'annotations-w-questions/' ,'labeled-questions/')

spdf = results_df_question_batch[results_df_question_batch['page'] == 'Spectrum_Science_Grade_3_80.jpeg']

amt_util.write_results_df(spdf, 'annotations-w-questions/', 'simpler-test-questions/')

## all pages from the latest batch
pages_to_review =pd.unique(consensus_results_df_complete['page'])
to_review = ['start_seq'] + pages_to_review.tolist()

# all pages from the latest batch with a no-consensus box
# pages_to_review =pd.unique(no_consensus_hits['page'])

pd.Series(to_review[1:]).to_pickle('rev_seq.pkl')

single_page = ['Daily_Science_Grade_2_Evan_Moor_33.jpeg']

problem_pages = pd.read_pickle('problem_pages.pkl').tolist()

to_review = ['start_seq'] + problem_pages

to_review = ['start_seq'] + single_page

suspect_subset = combined_results_df[combined_results_df['worker_id'].isin(bad_and_prolific_to_review)]
# suspect_subset = combined_results_df[combined_results_df['worker_id'].isin(['A3VE5OH94HYHET'])]
amt_util.write_results_df(suspect_subset, 'annotations-w-questions/' ,'labeled-questions2/')
pages_to_review = pd.unique(suspect_subset['page'])

In [150]:
## all pages from the latest batch
pages_to_review =pd.unique(results_df_question_batch['page'])
to_review = ['start_seq'] + pages_to_review.tolist()

In [ ]:
%%capture 
# all pages from the latest batch with a no-consensus box
suspect_subset = results_df_current_batch[results_df_current_batch['worker_id'].isin(suspect_workers[:5])]
amt_util.write_results_df(suspect_subset)
pages_to_review = pd.unique(suspect_subset['page'])

# Merging latest round of HITs into combined dataset  

## Load prior complete dataset if not in memory

In [17]:
data_pickled_dir = './store_hit_results_metadata/question_anno/group_latest_combined/' 

In [18]:
combined_results_df = pd.read_pickle(data_pickled_dir + 'complete_df.pkl')
combined_consensus_df = pd.read_pickle(data_pickled_dir + 'consensus_df.pkl')
combined_consensus_with_workerid_df = pd.read_pickle(data_pickled_dir + 'consensus_df_w_workers.pkl') 

In [19]:
amt_util.count_pages_in_df(combined_consensus_with_workerid_df)

## Updating full dataset

In [15]:
combined_results_df = results_df_question_batch
combined_consensus_df = consensus_results_df_complete
combined_consensus_with_workerid_df = consensus_with_workerid_df

## Working with full dataset

In [16]:
%%capture
no_consensus_hits = combined_consensus_df[combined_consensus_df['category'] == 'No Consensus']

In [964]:
flaw_rate = len(no_consensus_hits) / len(combined_consensus_df)
print 'text boxes without consensus are ' + '{0:0.2f}% '.format(flaw_rate * 100) + 'of the total'  

In [227]:
worker_quality_df[worker_quality_df['submitted'] > 50].sort_values('flaw_ratio', ascending= True).head(25)

In [205]:
bad_and_prolific_workers[:15]

After looking through the top few offenders, it's clear that 

1. the very worst intentionally submitted many blank pages

2. the rest didn't read the directions very closely

3. I'm comfortable rejecting the work of those with > 100 submissions

4. I'll ban the worst 15 from future HITs


In [49]:
amt_util.write_results_df(combined_consensus_df)

In [45]:
# all pages from the complete
pages_to_review =pd.unique(combined_consensus_df['page'])

In [165]:
# all pages from the complete dataset with a no-consensus box
pages_to_review =pd.unique(no_consensus_hits['page'])

In [208]:
%%capture 
# all pages from the latest batch with a no-consensus box
suspect_subset = combined_results_df[combined_results_df['worker_id'].isin(bad_and_prolific_to_review[10:15])]
amt_util.write_results_df(suspect_subset)
pages_to_review = pd.unique(suspect_subset['page'])

In [50]:
sampling_rate = 0.2
sample_size = int(len(pages_to_review) * sampling_rate)
sampled_pages_to_review = list(np.random.choice(pages_to_review, size= sample_size, replace=False))
print 'sampling ' + str(sample_size) + ' pages out of ' + str(len(pages_to_review))
to_review = ['start_seq'] + sampled_pages_to_review

In [51]:
amt_util.review_results(to_review)
print 'posting to review tool, navigate to http://localhost:8080/ to see the sampled consensus results'

# Worker Analysis

## Basic worker stats

In [209]:
consensus_with_workerid_df = amt_util.make_consensus_df_w_worker_id(results_df_question_batch, consensus_results_df_current_batch)

In [250]:
print 'number of unique workers:', pd.unique(results_df_question_batch['worker_id']).shape[0]

In [212]:
_ = results_df_question_batch['worker_id'].value_counts().hist(bins= 30)

### HIT duration for pricing

In [213]:
task_duration_seconds = []
for hit_id, assignments in assignment_results_current_batch.items():
    for assignment in assignments:
        hit_duration = dt_parse.parse(assignment.SubmitTime) - dt_parse.parse(assignment.AcceptTime)
        task_duration_seconds.append(hit_duration.seconds)

In [214]:
task_duration_series = pd.Series(task_duration_seconds)

In [94]:
_ = task_duration_series.hist(bins=30, log=True)

In [220]:
winsorized_durations = [t for t in task_duration_seconds if t < 150]
w_duration_series = pd.Series(winsorized_durations)

In [221]:
_ = pd.Series(w_duration_series).hist(bins=60)
plt.title('Worker task duration', fontsize=50, verticalalignment='bottom', color = b16_colors.b)
plt.ylabel("Number of Workers", fontsize=30, labelpad=10, color = b16_colors.b)
plt.xlabel("Seconds Spent on HIT", fontsize=30, labelpad=10, color = b16_colors.b)
plt.tick_params(axis='x', which='major', labelsize=20)
plt.tick_params(axis='y', which='major', labelsize=20)

In [224]:
dur_mode = w_duration_series.mode().values[0]
dur_median = w_duration_series.median()
print 'duration mode= ' + str(dur_mode)
print 'duration median= ' + str(dur_median)

### HIT pricing check

We'd like the effective hourly rate to be between 10-12 dollars. This is considered equitable in the world of mechanical turk.

In [241]:
hits_per_hour_high = 3600 / dur_mode
hits_per_hour_low = 3600 / dur_median
print 'effective hourly rate = $', hits_per_hour_low * static_params['amount'], 'to' , hits_per_hour_high * static_params['amount']

## Identifying high and low consensus workers

### high

In [244]:
worker_conflicts = consensus_with_workerid_df[consensus_with_workerid_df['category'] != consensus_with_workerid_df['consensus_category']]
all_worker_counts = results_df_question_batch['worker_id'].value_counts()
bad_worker_counts = worker_conflicts['worker_id'].value_counts()
worker_quality_df = pd.DataFrame([all_worker_counts, bad_worker_counts]).T
worker_quality_df.columns=['submitted', 'incorrect']
worker_quality_df['flaw_ratio'] = worker_quality_df['incorrect']/worker_quality_df['submitted']
good_workers = worker_quality_df.sort_values('flaw_ratio', ascending= True).index.tolist()

worker_quality_df.sort_values('flaw_ratio', ascending= True).head(10)

In [263]:
good_and_prolific_workers = worker_quality_df.sort_values('flaw_ratio', ascending= True).head(10).index.tolist()

I became concerned that I missed the perfect performers, but as I suspected perfect workers only did 1-2 HITs at most

In [66]:
# aw_set = set(all_worker_counts.index)
# bw_set = set(bad_worker_counts.index)
# flawless_workers = list(aw_set.difference(bw_set))
# all_worker_counts[all_worker_counts.index.isin(best_workers)]

### low

In [292]:
bad_and_prolific_workers = worker_quality_df.sort_values('flaw_ratio', ascending= False).head(25).sort_values('incorrect', ascending= False)

In [362]:
bad_and_prolific_to_review = list(bad_and_prolific_workers[:5].index)
really_bad_workers_to_review = list(really_bad_workers[:5].index)

In [294]:
bad_and_prolific_workers.head(15)

In [381]:
really_bad_workers = bad_and_prolific_workers[bad_and_prolific_workers['flaw_ratio'] > 0.5]
really_bad_workers

bad worker decisions-

1- didn't read directions

2- many blank pages

3- many blank pages

4- incomplete

the really bad workers are largely incomplete

## Email contact with workers

In [253]:
turkers_contacted_me = turkmail_util.get_latest_worker_communication()

In [257]:
turkers_contacted_me = [w_id[0] for w_id in turkers_contacted_me.values()]

In [258]:
turkers_contacted_me

In [269]:
good_set = set(good_and_prolific_workers)
bad_set = set(bad_and_prolific_to_review)
contact_set = set(turkers_contacted_me)

In [271]:
print good_set.intersection(contact_set)
print bad_set.intersection(contact_set)

## Messaging workers

In [889]:
subject = "More science book annotation HITs are available"
message = """
Hello, 

If you're receiving this message you were among the top performers on the first group HITs I submitted.
I've submitted another group of HITs, with more to follow in the next few days. 
I've added some additional instructions, and have removed the default instruction page (they can still be accessed by Read
Instruction Button).

Happy to get any feedback you might have for the new HITs.
"""

In [693]:
# _ = mturk.notify_workers(good_workers[:20], subject, message)

# **HIT end-of-life**

## Pickle latest results

In [208]:
#reset as needed
# gn = 2

In [207]:
next_group = gn + 1
group_n = '_' + str(gn) + '/'

temp_store_dir = './store_hit_results_metadata/question_anno/group'
try:
    os.mkdir(temp_store_dir + group_n)
except:
    OSError
    
result_file_name = 'hit_info.pkl'
assignment_file_name = 'assignment_info.pkl'
raw_results_file_name = 'raw_res.pkl'
complete_results_file = 'complete_df.pkl'
consensus_results_file = 'consensus_df.pkl'

amt_util.pickle_this(r_hits_current_batch, temp_store_dir + group_n + result_file_name)
amt_util.pickle_this(assignment_results_current_batch, temp_store_dir + group_n + assignment_file_name)
amt_util.pickle_this(results_df_question_batch, temp_store_dir + group_n + raw_results_file_name)
# results_df_current_batch.to_pickle(temp_store_dir + group_n + complete_results_file)
consensus_results_df_current_batch.to_pickle(temp_store_dir + group_n + consensus_results_file)
print 'saved HIT batch number ' + str(gn)
print 'now onto batch ' +str(next_group) 
gn = next_group

## Pickle combined dataset

In [110]:
temp_store_dir = './store_hit_results_metadata/question_anno/group'
group_n = '_latest_combined/'
try:
    os.mkdir(temp_store_dir + group_n)
except:
    OSError
    
complete_results_file = 'complete_df.pkl'
consensus_results_file = 'consensus_df.pkl'
consensus_results_file_w_workers = 'consensus_df_w_workers.pkl'

combined_results_df.to_pickle(temp_store_dir + group_n + complete_results_file)
combined_consensus_df.to_pickle(temp_store_dir + group_n + consensus_results_file)
combined_consensus_with_workerid_df.to_pickle(temp_store_dir + group_n + consensus_results_file_w_workers)

## Accepting and deleting HITs... careful with these

### rejecting HITS, banning bad workers

Uncomment only when ready to accept or delete hits

In [122]:
def reject_bad_assignments(rejected_assignment_ids, rejected_worker_ids):
    for assignment in rejected_assignment_ids:
        mturk.reject_assignment(assignment)
        
        rejection_message_subject = "One of your HITs was rejected"
        
        rejection_message = """
        Your HIT was rejected because it was either incomplete or largely incorrect.
        """
        
        mturk.notify_workers(rejected_worker_ids, rejection_message_subject, rejection_message)

In [155]:
workers_to_ban = bad_and_prolific_workers[:2].index.tolist()

In [156]:
workers_to_ban

In [157]:
number_rejected_assignments, number_rejected_workers = amt_util.reject_assignments(mturk, workers_to_ban, combined_consensus_with_workerid_df)
print 'rejecting ' + str(number_rejected_assignments) + ' assignments' + ' from ' + str(number_rejected_workers) + ' workers'

In [174]:
# assignments_to_accept = []
assignment_status = []
for hit_id, assignments in assignment_results_after_rejects.items():
    for assignment in assignments:
        assignment_status.append(assignment.AssignmentStatus)
        if assignment.AssignmentStatus == 'Submitted':
            assignments_to_accept.append(assignment)
status_series = pd.Series(assignment_status)

In [182]:
r_hits_after_rejects = amt_util.get_completed_hits(mturk)

In [183]:
assignment_results_after_rejects = amt_util.get_assignments(mturk, r_hits_after_rejects)

### accepting HITS

In [184]:
# amt_util.accept_hits(mturk, assignment_results_after_rejects)

### deleting all HITS

In [98]:
amt_util.delete_all_hits(mturk)

# End

In [47]:
images_to_test = ['1262.png', '1311.png', '1315.png', '1503.png', '193.png', '2822.png', '4106.png', '4138.png', '4930.png', '4947.png', '4949.png', '61.png','77.png']